## Exploratory Analysis
This section details of the exploratory analysis, data wrangling, and data management. 

## Notes


### The Grievance Dataset is merged with the source excel file by using Vlook-up. The dataset is attached for reference 

### Problem solving Approach is done in two methods 


### Summary 
#### The banks receives several thousands complaints filed by consumers concerning the practices of finanical companies. After reviewing each complaint, the Federal Bureau of US makes an anlaysis  regarding how the complaint was  resolved and grades the bank.

## Approach-1
####  Greivance Description is identified as a separate dataset to  classify the unique 86 Banks in the dataset based on the grievances of the customers, whether the bank is following the Federal Act of US for fair  practices in Banking . This is  a unsupervised problem statement .

## Approach-2 
#### This is a supervised ML problem with all the attributes apart from the "Grievance Description" mentioned above . The "Y" label here is the column  mergedfrom the Train.csv as "BankGrade". Here we need to identify as to how efficiently the bank has handled a customer grievance and based on the feedback of the resolution provided . After analysing and preprocessing the data we will prepare the model such that it can predict the accurate Grade  for the test data 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib

In [ ]:
import warnings 
warnings.filterwarnings("ignore")

### Visualization 

In [ ]:
import seaborn as sns
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE  # Helps visualize higher dimensions data

import squarify
import plotly
import plotly.graph_objs as go

%matplotlib inline
import plotly.offline as pyoff
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode, iplot, plot


# Make graphs prettier
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 700)
#pd.set_option('display.mpl_style', 'default')
matplotlib.style.use('ggplot')
# Make the fonts bigger
matplotlib.rc('figure', figsize=(14, 7))
matplotlib.rc('font', weight='bold', size=15)

### Reading the merged file in the notebook 

In [ ]:
data = pd.read_csv("GrievancesData_Train.csv", sep=",",header='infer')

In [ ]:
data

In [ ]:
data.shape

In [ ]:
# Describing the subset of data for values in each attributes
data.describe()

### We observe that BankID has 86 unique values which are similar to the train dataset. hence the merge of two files. this data includes "Bankgrade" as the last column 

### Before we begin let us calculate the Missing values on the datasets on each column 

In [ ]:
#This function computes the percentage of missing data per header name for a sub-frame of the CCD.
def missing_number(dataframe, header_name):
    count = 0
    for x in dataframe[header_name]:
        try:
            if np.isnan(x):
                count+=1
        except TypeError:
            pass
    return (count)



#This function counts unique elements in columns
def count_unique(dataframe, header_name):
    z = dataframe[header_name].unique()
    return sum(1 for v in z if pd.notnull(v))

In [ ]:
r = []
for col in data.columns:
    r.append([col, missing_number(data,col),count_unique(data,col)])
missing_data = pd.DataFrame(r, columns=['Column Name', 'Missing Val', 'Unique'])
print(missing_data)

### We can deduce that the above data has 121 missing values in the dataset on Column State , which implies that 121 banks do not have their State information 

In [ ]:
for column in data.columns:
    data[column].fillna(data[column].mode()[0], inplace=True)

##### For experimentation and trial and error purposes , lets make a duplicate copy of the Non NA value data and use that for the analysis , instead of ruining the main data 

In [ ]:
data_set = data

#### It is observed that for 86 unique banks there are 53680 grievances , which implies that a single bank has repeated grievances and irrespective of the grievance the "BankGrade" for all the comments on bank is either of the ('Outstanding','Defecient ','Satisfactory'). Hence if we keep the bankID as an attribute for ML it will explicitly consider the many similar charachteristic between BankID and its BankGrade and provide the same result in test data , the result will also be biased. It will no doubt provide a great accuracy but it will not be a "wise" metric to evaluate a bank 

In [ ]:
## Only interested in subset of data without consumer complaints
##Taking subset of the data

df = data_set[['GrievanceID','BankID','State','DateOfGrievance','Grievance_Category','LineOfBusiness','ResolutionComments','Disputed','DateOfResolution','BankGrade']]
df.head(10)

In [ ]:
df.columns

In [ ]:
#Type casting
df['DateOfGrievance'] = pd.to_datetime(df['DateOfGrievance'], format='%m/%d/%Y')
df['DateOfResolution'] = pd.to_datetime(df['DateOfResolution'], format='%m/%d/%Y')



In [ ]:
#Check size and dtypes
print(df.shape)
print(data.dtypes)

##### Calculate the time bank has taken to resolve the complaint 

In [ ]:
from datetime import timedelta

df['Processing_time'] = df['DateOfResolution']-df['DateOfGrievance']
df.loc[df['Processing_time']<timedelta(days=0),'DateOfResolution'] = df['DateOfGrievance']

In [ ]:
df['Year'] = pd.to_datetime(df['DateOfGrievance'], format = '%d%b%Y')
df['Year']= df['Year'].dt.year

In [ ]:
df['Processing_time'] = df['Processing_time'].astype('timedelta64[D]').astype(int)

In [ ]:
df

#### Graph to find out which companies received the most complaints 

In [ ]:
grp = df.groupby('BankID')
grp = grp.size().sort_values(ascending=False)
#top10 = grp[10::-1]
top10 = (grp[:10])[::-1]
top10.plot(kind='barh',title='Most Common Banks to Complaint Against')

In [ ]:
grp = df.pivot_table(index='ResolutionComments',columns='Disputed',values='DateOfGrievance',aggfunc='count')
grp

In [ ]:
df['ResolutionComments'].value_counts().plot(title="Category_distibution", kind='pie')
plt.show()

### The above pivot table gives us a general idea that Banks which gave the status as "Closed with explanation" were challenged or disputed again most number of times . An assumption would be that, customer was not given a correct explanation , also one observation can be made that maximum cases by banks were tried to resolve by "Explanation" only 

In [ ]:
## The below plot tells us that 2013 saw the maximum number of complaints and a decreasing trend in complaints post 2014


grp = df.groupby('Year')
grp.size().plot(kind='bar',title='Volume of Complaints by Year (2011/2016 partial)')

### Let us get a general idea of Complaints data as per State

In [ ]:
new_data01 = df.groupby(["State"]).size().reset_index(name="Count_of_Complaints")
new_data01.head(5)

In [ ]:
## Study of Complaints based on Line of Business of each banks 
df['LineOfBusiness'].value_counts().plot(kind='bar')

### Nature of complaint in each category of grievance 

#### Below graph explains about the grievances received in each category of grievance specified in dataset

In [ ]:
fig=plt.figure(figsize=(18, 40), dpi= 80, facecolor='w', edgecolor='k')
count_plot = sns.countplot(y = df['Grievance_Category'],order=df['Grievance_Category'].value_counts().index)
count_plot.set_ylabel('Grievance_Category')
count_plot.set_title('No. of Grievances in each Sector')
plt.xlabel('Num_of_Grievances')


##### The above chart helps us to identify that based on Grievance category , Out of the total 67  unique categories,  roughly 30 are major contributors to highlight the grievances amongst consumers

#### The below date tells us the percent rating of banks in an overview

We observe that satisfactory rating of resolution of banks is 64% and that of defecient is 26%

In [ ]:
label = pd.DataFrame(df.BankGrade.value_counts())
label.columns = ['BankGrade_cnt']
label['Bankrating'] = label.BankGrade_cnt/df.shape[0]

base = 0.972434

label

** Product **

We could see that how the banks are faring based on their various product lines 

In [ ]:
f,ax=plt.subplots(figsize=(18,8))
sns.countplot(y = 'LineOfBusiness', hue='BankGrade', data=data)
plt.show()

#### the rate of defecienct is obviously higher in credit card related issues , this could probably be the non-payment issue by customers 

### Since the columns dont add up enough to give data for ML model , frequency of occurence of various categorical columns based on the grading of banks might help . 

### Since the model experiment will be carried out twice , Once by using BankID and once by removing BankID to avoid Bias as mentioned before , we might need these frequency columns to provide the ML algorithms sufficient data that they do not overfit

In [ ]:
def freq_by_cat(data, cat):
    '''
    Input: data is the whole data set;
           cat is the categorical feature you want to calculate the frequency of each category inside this feature.
    Output: a dataframe
    '''
    
    cat = str(cat)
    # Calculate the frequency of each cat
    freq = data.groupby('%s'%cat).count()/53680
    # reset index
    freq['%s'%cat] = freq.index
    freq.columns = ['freq_by_%s'%cat, '%s'%cat]
    freq = freq.reset_index(drop=True)
    
    return freq

** Issue **

New features:

- freq_by_issue: Frequency of issue

In [ ]:
Issue = df[['Grievance_Category', 'BankGrade']]
freq_by_issue = freq_by_cat(Issue, 'Grievance_Category')
#freq_by_issue

In [ ]:
# One new feature: freq_by_issue
df = pd.merge(df, freq_by_issue, on='Grievance_Category')

In [ ]:
df.head()

** LineOfBusiness **

New features:

- freq_by_business: Frequency of business

In [ ]:
Business = df[['LineOfBusiness', 'BankGrade']]

freq_by_business = freq_by_cat(Business, 'LineOfBusiness')

In [ ]:
# One new feature: freq_by_product
df = pd.merge(df, freq_by_business, on='LineOfBusiness')

** State **

New features:

- freq_by_stateID: Frequency of State

In [ ]:
State = df[['State', 'BankGrade']]

freq_by_State = freq_by_cat(State, 'State')

In [ ]:
# One new feature: freq_by_state
df = pd.merge(df, freq_by_State, on='State')

## One hot encoding  categorization of values to numerical

In [ ]:
df.BankGrade[df.BankGrade == 'deficient'] = 0
df.BankGrade[df.BankGrade == 'satisfactory'] = 1
df.BankGrade[df.BankGrade == 'outstanding'] = 2

df.BankGrade = df.BankGrade.astype('int')

In [ ]:
dummies = pd.get_dummies(df[['State','Grievance_Category','LineOfBusiness','ResolutionComments','Disputed']])

In [ ]:
df = df.drop(['GrievanceID','BankID','DateOfGrievance','DateOfResolution','Year'],axis =1 , inplace = True)

In [ ]:
df = pd.concat([df,dummies],1,join = 'inner')
df.head(3)

### Model

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
import sklearn.metrics as skm
from sklearn.preprocessing import MaxAbsScaler
from sklearn.pipeline import Pipeline
from scipy import interp
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.svm import SVC